**Задание**


Реализовать суммаризацию текста


In [2]:
# Загрузка данных

# !wget -q https://www.dropbox.com/s/43l702z5a5i2w8j/gazeta_train.txt
# !wget -q https://www.dropbox.com/s/k2egt3sug0hb185/gazeta_val.txt
# !wget -q https://www.dropbox.com/s/3gki5n5djs9w0v6/gazeta_test.txt

In [3]:
# Чтение данных

import json
import random

def read_gazeta_records(file_name, shuffle=True, sort_by_date=False):
    assert shuffle != sort_by_date
    records = []
    with open(file_name, "r") as r:
        for line in r:
            records.append(json.loads(line))
    if sort_by_date:
        records.sort(key=lambda x: x["date"])
    if shuffle:
        random.shuffle
    return records

In [4]:
# train_records = read_gazeta_records("gazeta_train.txt")
# val_records = read_gazeta_records("gazeta_val.txt")
test_records = read_gazeta_records("gazeta_test.txt")

In [5]:
import pandas as pd

dataset = pd.DataFrame(test_records)
dataset.head(3)

,url,text,title,summary,date
0,https://www.gazeta.ru/science/2020/02/14_a_129...,Американское аэрокосмическое агентство NASA ог...,"Венера, Ио или Тритон: куда полетит NASA","В NASA назвали четыре миссии в дальний космос,...",2020-02-14 16:39:11
1,https://www.gazeta.ru/social/2020/02/28/129806...,Около 11 тысяч зрителей увидели все самое лучш...,«Люди в Бурятии очень талантливые»,25 и 26 февраля в Кремлевском дворце съездов п...,2020-02-28 10:44:13
2,https://www.gazeta.ru/politics/2019/11/07_a_12...,7 ноября в Белоруссии прошли выборы членов сов...,Вспомнить СССР: как Лукашенко провел выборы,В Белоруссии в день годовщины Октябрьской рево...,2019-11-07 19:55:08


In [6]:
dataset = dataset.drop(['url', 'date'], axis=1)

In [7]:
# Возмем кусок датасета для теста 

dataset = dataset[:30]

### Используем Модель MBARTRuSumGazeta

In [8]:
# Загружаем модель

from transformers import MBartTokenizer, MBartForConditionalGeneration

model_name = "JuliaPolushina/mbart_ru_sum_gazeta"
tokenizer = MBartTokenizer.from_pretrained(model_name)
model = MBartForConditionalGeneration.from_pretrained(model_name)

In [9]:
# Посмотрим как работает модель

article_text = dataset['text'][5]

input_ids = tokenizer(
    [article_text],
    max_length=600,
    padding="max_length",
    truncation=True,
    return_tensors="pt",
)["input_ids"]

output_ids = model.generate(
    input_ids=input_ids,
    no_repeat_ngram_size=4
)[0]

summary = tokenizer.decode(output_ids, skip_special_tokens=True)

print('Predicted summary: ', summary)
print('Real summary: ', dataset['summary'][5])

/Users/aleksandrkrylov/anaconda3/lib/python3.9/site-packages/transformers/generation/utils.py:1353: UserWarning: Using `max_length`'s default (200) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Predicted summary:  В Дании назвали «новым кулаком» страны в Арктике головной патрульный корабль ледового класса «Иван Папанин», который спустили на воду 25 октября на Адмиралтейских верфях. По мнению датского журналиста Питера Суппли Бенсона, новость о спуске боевого корабля на воду в Санкт-Петербурге получила международный резонанс. Он также отметил, что Дания инвестирует в Арктику слишком мало средств и из-за этого отстает от России, которая тем временем создает новые ледоколы.
Real summary:  Датского журналиста обеспокоила активность России в Арктике, как и новый головной патрульный корабль ледового класса «Иван Папанин». Он был спущен на воду в конце октября и обратил на себя внимание представителей СМИ тем, что может быть вооружен как пушками, так и ракетами, способными стрелять по другим кораблям.


In [10]:
# Генерация summary для всего датасета

data = dataset['text']

summaries = []

for row in data:
    input_ids = tokenizer(
        [row],
        max_length=600,
        padding="max_length",
        truncation=True,
        return_tensors="pt",
    )["input_ids"]

    output_ids = model.generate(
        input_ids=input_ids,
        no_repeat_ngram_size=4
    )[0]

    summary = tokenizer.decode(output_ids, skip_special_tokens=True)
    summaries.append(summary)

/Users/aleksandrkrylov/anaconda3/lib/python3.9/site-packages/transformers/generation/utils.py:1353: UserWarning: Using `max_length`'s default (200) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [11]:
dataset['pred_summary'] = summaries

In [12]:
dataset.sample(3)

,text,title,summary,pred_summary
3,Народная артистка РСФСР Надежда Бабкина в инте...,«Он очень переживал»: Бабкина об отношениях с ...,Народная артистка РСФСР Надежда Бабкина в инте...,Народная артистка РСФСР Надежда Бабкина расска...
18,"Американскому круизному лайнеру Westerdam, на ...",«Нет больных? Откуда знаете?» Лайнер из США ни...,"Круизный лайнер Westerdam, принадлежащий амери...","Американскому круизному лайнеру Westerdam, на ..."
6,Выступление главы российской делегации Петра Т...,"«Смотрите, что у вас происходит»: как прервали...",Активист сорвал выступление главы делегации РФ...,Выступление главы российской делегации Петра Т...


In [13]:

from datasets import load_metric

rouge_score = load_metric("rouge")

In [14]:
predictions = dataset['pred_summary']
references = dataset['summary']

In [15]:
# Оценка по Rouge

sum_scores = rouge_score.compute(
    predictions=[predictions], references=[references]
)
sum_scores

{'rouge1': AggregateScore(low=Score(precision=0.30952380952380953, recall=0.65, fmeasure=0.41935483870967744), mid=Score(precision=0.30952380952380953, recall=0.65, fmeasure=0.41935483870967744), high=Score(precision=0.30952380952380953, recall=0.65, fmeasure=0.41935483870967744)),
 'rouge2': AggregateScore(low=Score(precision=0.14634146341463414, recall=0.3157894736842105, fmeasure=0.19999999999999998), mid=Score(precision=0.14634146341463414, recall=0.3157894736842105, fmeasure=0.19999999999999998), high=Score(precision=0.14634146341463414, recall=0.3157894736842105, fmeasure=0.19999999999999998)),
 'rougeL': AggregateScore(low=Score(precision=0.30952380952380953, recall=0.65, fmeasure=0.41935483870967744), mid=Score(precision=0.30952380952380953, recall=0.65, fmeasure=0.41935483870967744), high=Score(precision=0.30952380952380953, recall=0.65, fmeasure=0.41935483870967744)),
 'rougeLsum': AggregateScore(low=Score(precision=0.30952380952380953, recall=0.65, fmeasure=0.419354838709677